In [1]:
import os

In [2]:
for fls in os.walk('./Data/'):
    books = fls[2]

In [3]:
def read_data(books):    
    train = ''
    test = ''
    for i in books:
        print(i)
        path = './Data/' + i
        f = open(path, encoding = "utf-8")
        data_ = f.read()
        f.close()
        train = train + '\n' + data_[:int(len(data_)*0.75)]
        test = test + '\n' + data_[int(len(data_)*0.75):]
    f = open('./Data/train_dataset.txt', 'w', encoding="utf-8")
    f.write(train)
    f = open('./Data/test_dataset.txt', 'w', encoding="utf-8")
    f.write(test)
    f.close()
    

In [4]:
read_data(books)

Руслан и Людмила.txt
Сказка о золотом петушке.txt
Сказка о мёртвой царевне.txt
Сказка о попе и работнике его Балде.txt
Сказка о царе Салтане.txt


In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
train_path = './Data/train_dataset.txt'
test_path = './Data/test_dataset.txt'

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=64)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=64)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


C:\Users\Mike PK\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [9]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

In [ ]:
# model.save_pretrained("./gpt2-sv",push_to_hub = False)

In [10]:
training_args = TrainingArguments(
    output_dir="./gpt2-sv", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    evaluation_strategy = "steps",
    eval_steps = 100, # Number of update steps between two evaluations.
#     save_steps=100, # after # steps model is saved
    warmup_steps=50,# number of warmup steps for learning rate scheduler
    logging_strategy = "steps",
    logging_steps = 100
    )

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [22]:
trainer.train()

***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Step,Training Loss,Validation Loss
100,1.834400,3.253605
200,1.858000,3.253605
300,1.844000,3.253605


***** Running Evaluation *****
  Num examples = 165
  Batch size = 4
***** Running Evaluation *****
  Num examples = 165
  Batch size = 4
***** Running Evaluation *****
  Num examples = 165
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=375, training_loss=1.8400179036458333, metrics={'train_runtime': 2799.6927, 'train_samples_per_second': 0.536, 'train_steps_per_second': 0.134, 'total_flos': 174131380224000.0, 'train_loss': 1.8400179036458333, 'epoch': 3.0})

In [ ]:
#  trainer.save_model()

In [23]:
prefix = "Белка прыгала сама"

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}



In [24]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+100, 
    repetition_penalty=5., 
    temperature=0.7,
    num_beams=6,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Белка прыгала сама не зная куда.
«Куда ты, белочка? — молвила ей царица: —
Попляши-ка под мою дудочку».
Но белка в ответ:
«Я песенку пою одна,
А плясать мне велят все вокруг».
Царь к ней с поклоном подходит,
Говорит: «Добрый день, девица!
Что же ты тиха, как день ненастный?
Опечалена чем-нибудь?»


In [25]:
model.save_pretrained("./model",push_to_hub = False)

Configuration saved in ./model\config.json
Model weights saved in ./model\pytorch_model.bin
